In [ ]:
# Install Seurat if it is not already installed
install.packages('Seurat')
library(Seurat)

In [ ]:
# Load required libraries
library(dplyr)
library(Seurat)
library(patchwork)
library(DoubletFinder)
library(DropletUtils)
library(Matrix)

In [ ]:
# Pick one of the below data loading cells, depending on what form of 
# the data you are starting from

# No Preprocessing: start from CellRanger count output
    # Replace the data.dir paths with the paths to your data
    # Replace the "samp1", "samp2", etc with the names of your data (i.e. "PFC-HT1")
    # Replace "projectName" with the name of your project (i.e. "geneX-brainRegion")

samp1 <- "samp1"
samp1.data <- Read10X(data.dir="parentDir/data/mkfastq/projectName-samp1/projectName-samp1/outs/filtered_feature_bc_matrix/")
samp1 <- CreateSeuratObject(counts=samp1.data, project = "projectName_samp1", min.cells = 3, min.features =200)
samp2 <- "samp2"
samp2.data <- Read10X(data.dir="parentDir/data/mkfastq/projectName-samp2/projectName-samp2/outs/filtered_feature_bc_matrix/")
samp2 <- CreateSeuratObject(counts=samp2.data, project = "projectName_samp2", min.cells = 3, min.features =200)
samp3 <- "samp3"
samp3.data <- Read10X(data.dir="parentDir/data/mkfastq/projectName-samp3/projectName-samp3/outs/filtered_feature_bc_matrix/")
samp3 <- CreateSeuratObject(counts=samp3.data, project = "projectName_samp3", min.cells = 3, min.features =200)

# Merge samples into one Seurat object
object <- merge(samp1, y=c(samp2,samp3), add.cell.ids=c("samp1","samp2","samp3"), project="projectName")
object
head(object@meta.data)

# Add genotype to the metadata
    # put your sample names (called "orig.ident") in the from list and the genotypes of 
    # each sample in the to list
model <- plyr::mapvalues(
    x = object@meta.data$orig.ident, 
    from = c("samp1", "samp2", "samp3"), 
    to = c("HT", "KO", "WT")
)
head(model)
object@meta.data$model <- model
head(object@meta.data)

In [ ]:
# Downsampling: start from CellRanger aggr output

object <- "projectName"
object.data <- Read10X(data.dir="parentDir/data/aggr/projectName/outs/filtered_feature_bc_matrix/")
object <- CreateSeuratObject(counts=object.data, project = "projectName", min.cells = 3, min.features =200)

# Extract barcodes from cell names for sample mapping

barcodes <- row.names(object@meta.data)
nums <- read.table(text=barcodes, sep="-")
head(nums)

# Map barcode numbers to sample names & genotypes to samples

sampleName <- plyr::mapvalues(
    x = nums$V2, 
    from = c(1,2,3), 
    to = c("samp1","samp2","samp3")
)
head(sampleName)

# Add genotype to the metadata
    # put your sample names (called "orig.ident") in the from list and the genotypes of 
    # each sample in the to list
model <- plyr::mapvalues(
    x = object@meta.data$orig.ident, 
    from = c("samp1", "samp2", "samp3"), 
    to = c("HT", "KO", "WT")
)
head(model)

object@meta.data$orig.ident <- sampleName
object@meta.data$model <- model
head(object@meta.data)

In [ ]:
# Ambient Removal: start with CellBender output
    # Replace the filename paths with the paths to each sample's filtered.h5 file
    # Replace the "samp1", "samp2", etc with the names of your data (i.e. "PFC-HT1")
    # Replace "projectName" with the name of your project (i.e. "geneX-brainRegion")

samp1 <- "samp1"
samp1.data <- Read10X_h5(filename="parentDir/data/ambientRNA/SAMPLE1/SAMPLE1.results_filtered.h5", use.names=TRUE)
samp2 <- "samp2"
samp2.data <- Read10X_h5(filename="parentDir/data/ambientRNA/SAMPLE2/SAMPLE2.results_filtered.h5", use.names=TRUE)
samp3 <- "samp3"
samp3.data <- Read10X_h5(filename="parentDir/data/ambientRNA/sample3/sample3.results_filtered.h5", use.names=TRUE)

object <- merge(samp1, y=c(samp2,samp3), add.cell.ids=c("samp1","samp2","samp3"), project="projectName")
object
head(object@meta.data)

# Add genotype to the metadata
    # put your sample names (called "orig.ident") in the from list and the genotypes of 
    # each sample in the to list
model <- plyr::mapvalues(
    x = object@meta.data$orig.ident, 
    from = c("samp1", "samp2", "samp3"), 
    to = c("HT", "KO", "WT")
)
head(model)
object@meta.data$model <- model
head(object@meta.data)

In [ ]:
# Checkpoint 1: save merged raw data

#saveRDS(object, "GENE_AGE_aggrObject_DATE.rds")
object <- readRDS("GENE_AGE_aggrObject_DATE.rds")

In [ ]:
# Reassign idents to sample names

as.data.frame(table(object@meta.data$orig.ident))
as.data.frame(table(object@meta.data$model))
Idents(object) <- object@meta.data$orig.ident

In [ ]:
# Vis QC metrics as violin plot
VlnPlot(object, features = c("nFeature_RNA", "nCount_RNA"), ncol=2, pt.size=0)

In [ ]:
# Filter out cells with too few features 
object <- subset(object, subset = nFeature_RNA > 500)
# Normalizing data
object <- NormalizeData(object, normalization.method = "LogNormalize", scale.factor = 10000)
# Post filtering, normalization violin plot of QC metrics
VlnPlot(object, features = c("nFeature_RNA", "nCount_RNA"), ncol=2, pt.size=0)

In [ ]:
# Feature selection (via identification of highly variable features)
object <- FindVariableFeatures(object, selection.method = "vst", nfeatures = 2000)

# Top ten most highly variable genes
top10 <- head(VariableFeatures(object), 10)
top10

# plot variable features with/without labels
plot1 <- VariableFeaturePlot(object)
plot2 <- LabelPoints(plot=plot1, points = top10, xnudge=0, repel=TRUE)
plot1
plot2

In [ ]:
# Scaling data
    # linear transformation (scaling): preprocessing step prior to dimension reduction
all.genes <- rownames(object)
object <- ScaleData(object, features = all.genes)

In [ ]:
# Linear Dimensional Reduction (using PCA)
object <- RunPCA(object, features = VariableFeatures(object=object))

# Look at just top 5 genes for each principle component (PC)
print(object[["pca"]], dims= 1:5, nfeatures = 5)

VizDimLoadings(object, dims = 1:2, reduction="pca")
DimPlot(object, reduction="pca", group.by="model")
DimPlot(object, reduction="pca")
ElbowPlot(object)

In [ ]:
pdf("plots/project_pcaBySample_date.pdf")
DimPlot(object, reduction="pca")
dev.off()

pdf("plots/project_pcaByGenotype_date.pdf")
DimPlot(object, reduction="pca", group.by="model")
dev.off()

In [ ]:
# Cluster Cells
object <- FindNeighbors(object, dims=1:X)
object <- FindClusters(object, resolution = Y)

# Run non-linear dimension reduction (UMAP/tSNE)
object <- RunUMAP(object, dims=1:X)

In [ ]:
# Look at UMAP clusters before running doublet identification

DimPlot(object, reduction="umap", label=TRUE)
DimPlot(object, reduction="umap", group.by="model")
DimPlot(object, reduction="umap", group.by="orig.ident")

In [ ]:
# Write counts to matrix for double removal

writeMM(object@assays$RNA@counts,"counts.txt")

# Stop here to run RunScrublet.py
 # Use the below commands to run this script. Please note, you must install Scrublet (preferably in a conda environment) before running the script.
 # $ use Anaconda3/n # $ source activate Scrublet_environment/n # $ Python RunScrublet.py counts.txt output.txt

In [ ]:
# Import scrublet results, add to metadata and look at score distribution

object@meta.data["scrublet"] = scan("output.txt")
hist(object@meta.data$scrublet, col="gray")

In [ ]:
# Assign doublet label to cells above scrublet score threshhold

object@meta.data["doublet"]="singlet"
object@meta.data[object@meta.data[,"scrublet"]>THRESHOLD,"doublet"]="doublet"

In [ ]:
# Assess number of doublets and their distribution

table(object@meta.data$doublet)
table(object@meta.data$orig.ident)
FeaturePlot(object, features="scrublet")
pdf("plots/dataCleaning/doublets.pdf")
FeaturePlot(object, features="scrublet")
dev.off()
pdf("plots/dataCleaning/labeledDoublets.pdf")
FeaturePlot(object, features="scrublet", label=TRUE)
dev.off()

In [ ]:
# Sanity check: see if clusters with high % doublets coexpress markers for multiple cell types
FeaturePlot(object, features = c("Gad1"), label=TRUE) # Inhibitory Neurons
FeaturePlot(object, features = c("Slc17a7"), label=TRUE) # Excitatory Neurons
FeaturePlot(object, features = c("Csf1r"), label=TRUE) # Microglia
FeaturePlot(object, features = c("Pdgfra"), label=TRUE) # OPC
FeaturePlot(object, features = c("Plp1"), label=TRUE) # ODC
FeaturePlot(object, features = c("Slc1a3"), label=TRUE) # Astrocytes
FeaturePlot(object, features = c("Flt1"), label=TRUE) # Endothelial
FeaturePlot(object, features = c("Bgn"), label=TRUE) # Other Vascular
FeaturePlot(object, features = c("Bnc2"), label=TRUE) # Fibroblast

In [ ]:
# Label small clusters with high % doublets and remove them
doublet_clusters <- c()
object@meta.data[object@active.ident %in% doublet_clusters,"doublet"]="doublet"
DimPlot(object, reduction="umap", group.by="doublet")
pdf("plots/dataCleaning/doubletClassFinal.pdf")
DimPlot(object, reduction="umap", group.by="doublet")
dev.off()

#object=subset(object,doublet=="singlet")

In [ ]:
# Check removed clusters
pdf("plots/dataCleaning/removedDoublets.pdf")
FeaturePlot(object, features="scrublet")
dev.off()
pdf("plots/dataCleaning/labeledremovedDoublets.pdf")
FeaturePlot(object, features="scrublet", label=TRUE)
dev.off()

In [ ]:
# Checkpoint 2: Pre-labeled object with doublets removed

saveRDS(object, "project_preLabel_date.rds")
#object <- readRDS("project_preLabel_date.rds")

In [ ]:
# Evaluate cell type markers and determine cluster cell types

DimPlot(object, reduction="umap", label=TRUE)
DimPlot(object, reduction="umap", label=TRUE) + NoLegend()

In [ ]:
# neurons (Excitory and inhibitory) - 
FeaturePlot(object, features = c("Snap25"), label=TRUE)
FeaturePlot(object, features = c("Rbfox3"), label=TRUE)

In [ ]:
# Inhibitory Neurons 
FeaturePlot(object, features = c("Gad1"), label=TRUE)
FeaturePlot(object, features = c("Gad2"), label=TRUE)

In [ ]:
# Excitatory neurons 
FeaturePlot(object, features = c("Slc17a6"), label=TRUE)
FeaturePlot(object, features = c("Slc17a7"), label=TRUE)
FeaturePlot(object, features = c("Neurod6"), label=TRUE)

In [ ]:
# Microglia - 
FeaturePlot(object, features = c("Csf1r"), label=TRUE)

In [ ]:
# OPC - 
FeaturePlot(object, features = c("Pdgfra"), label=TRUE)
FeaturePlot(object, features = c("Olig1"), label=TRUE)

In [ ]:
# ODC (mODC and nODC) - 
FeaturePlot(object, features = c("Plp1"), label=TRUE)
FeaturePlot(object, features = c("Mbp"), label=TRUE)
FeaturePlot(object, features = c("Mobp"), label=TRUE)
FeaturePlot(object, features = c("Olig1"),label=TRUE)

In [ ]:
# Astrocytes -
FeaturePlot(object, features = c("Slc1a3"), label=TRUE)
FeaturePlot(object, features = c("S100b"), label=TRUE)
FeaturePlot(object, features = c("Gfap"), label=TRUE)

In [ ]:
# Endothelial
FeaturePlot(object, features = c("Flt1"), label=TRUE)

In [ ]:
# Other Vascular Markers (Pericytes, Smooth Muscle, etc) -
FeaturePlot(object, features = c("Bgn"), label=TRUE)
FeaturePlot(object, features = c("Vtn"), label=TRUE)

In [ ]:
# Fibroblast-like - 
FeaturePlot(object, features = c("Bnc2"), label=TRUE)

In [ ]:
# Assigning Cell Type Identity to clusters
    # For this example, we know what markers map to each cluster. Look for assignment algorithm (like SingleR)
new.cluster.ids2 <- c()
names(new.cluster.ids2) <- levels(object)
object <- RenameIdents(object, new.cluster.ids2)
DimPlot(object, reduction = "umap", label = TRUE)

In [ ]:
# Save commonly needed UMAP plots

pdf("plots/project_legend_cellTypes_date.pdf")
DimPlot(object, reduction = "umap", label = FALSE)
dev.off()

pdf("plots/project_cellTypes_date.pdf")
DimPlot(object, reduction = "umap", label = TRUE) + NoLegend()
dev.off()

pdf("plots/project_umapGenotype_date.pdf")
DimPlot(object, group.by="model")
dev.off()

In [ ]:
# Create cell type distribution plot by genotype

library(ggplot2)
library("RColorBrewer")
object@meta.data$cellType = Idents(object)
pt <- as.data.frame(table(object$cellType, object$model))
pt$cellType <- as.character(pt$Var1)
pt$Model <- pt$Var2
dim(pt)

# Assuming two genotypes
pt$Percentage <- c(pt$Freq[1:9]/sum(pt$Freq[1:9]), pt$Freq[10:18]/sum(pt$Freq[10:18]))
# Assuming three genotypes
pt$Percentage <- c(pt$Freq[1:9]/sum(pt$Freq[1:9]), pt$Freq[10:18]/sum(pt$Freq[10:18]), pt$Freq[19:27]/sum(pt$Freq[19:27]))
pt

my_levels2 <- c("Excitatory Neurons", "Inhibitory Neurons", "Astrocytes", "ODC", 
                "Fibroblast-like", "Microglia", "OPC", "Endothelial", "Other Vascular")
factor(pt$cellType, levels= my_levels2)
pt$cellType <- factor(pt$cellType, levels= my_levels2)
#pdf("plots/project_ascendingPercentageCelltype_date.pdf")
ggplot(pt, aes(x = Model, y = Percentage, fill = cellType, label=round(Percentage,2))) +
    geom_bar(stat="identity") +
    geom_text(size=3, position=position_stack(vjust=0.5))
#dev.off()

In [ ]:
object@meta.data$cellType = Idents(object)
pt <- as.data.frame(table(object$cellType, object$orig.ident))
pt$cellType <- as.character(pt$Var1)
replicate <- plyr::mapvalues(
    x = pt$Var2, 
    from = c(), 
    to = c()
)
replicate
pt$Sample <- replicate
pt
dim(pt)
# Adjust to align with number of samples
pt$Percentage <- c(pt$Freq[1:9]/sum(pt$Freq[1:9]), 
                   pt$Freq[10:18]/sum(pt$Freq[10:18]), 
                   pt$Freq[19:27]/sum(pt$Freq[19:27]),
                   pt$Freq[28:36]/sum(pt$Freq[28:36]),
                   pt$Freq[37:45]/sum(pt$Freq[37:45]),
                   pt$Freq[46:54]/sum(pt$Freq[46:54]),
                   pt$Freq[55:63]/sum(pt$Freq[55:63]),
                   pt$Freq[64:72]/sum(pt$Freq[64:72]),
                   pt$Freq[73:81]/sum(pt$Freq[73:81]),
                   pt$Freq[82:90]/sum(pt$Freq[82:90]),
                   pt$Freq[91:99]/sum(pt$Freq[91:99])
                  )
pt

my_levels2 <- c("Excitatory Neurons", "Inhibitory Neurons", "Astrocytes", "ODC", 
                "Fibroblast-like", "Microglia", "OPC", "Endothelial", "Other Vascular")
factor(pt$cellType, levels= my_levels2)
pt$cellType <- factor(pt$cellType, levels= my_levels2)
#pdf("plots/project_replicateAscendingPercentageCelltype_date.pdf")
ggplot(pt, aes(x = Sample, y = Percentage, fill = cellType, label=round(Percentage,2))) +
    geom_bar(stat="identity") +
    geom_text(size=3, position=position_stack(vjust=0.5))
#dev.off()

In [ ]:
# Checkpoint 3: Post-labeled object 

saveRDS(object, "project_postLabel_date.rds")
#object <- readRDS("project_postLabel_date.rds")